In [2]:
import pandas as pd
import duckdb as db
import json

### Download the geonames.org alternative Name Database

- Go to https://download.geonames.org/export/dump/
- Download the `alternateNamesV2.zip`

In [5]:
alternate_names_columns = ['alternateNameId', 'geonameid', 'isolanguage', 'alternate name', 'isPreferredName', 'isShortName', 'isColloquial', 'isHistoric', 'from', 'to']
alternate_names_dtypes = {'alternateNameId': int, 'geonameid': int, 'isolanguage': str, 'alternate name': str, 'isPreferredName':str, 'isShortName':str, 'isColloquial':str, 'isHistoric':str, 'from':str, 'to': str}
filename = './alternateNamesV2.txt'
alname_df = pd.read_csv(filename, sep='\t', header=None, names=alternate_names_columns, dtype=alternate_names_dtypes)
alname_df.fillna('', inplace=True)

In [6]:
alname_df.head()

,alternateNameId,geonameid,isolanguage,alternate name,isPreferredName,isShortName,isColloquial,isHistoric,from,to
0,1284819,2994701,,Roc Mélé,,,,,,
1,1284820,2994701,,Roc Meler,,,,,,
2,1291197,3017832,,Pic de les Abelletes,,,,,,
3,4290387,3017832,,Pic de la Font-Nègre,,,,,,
4,1291198,3017833,,Estany de les Abelletes,,,,,,


### Processing the alternative/foreign language database

Preparsing
- ISO language code are 2 letters (Major filter)
- We need to remove where ISO language values are empty
- We need to remove where Geoname ID values are empty
- We need to remove where Alternative name values are empty

In [7]:
alname_df_all_values_imported = db.sql("""SELECT *
FROM alname_df
WHERE geonameid is not null
AND isolanguage !=''
AND "alternate name" !='' 
AND LENGTH(isolanguage) <= 2
""").df()
alname_df_all_values_imported.head()

,alternateNameId,geonameid,isolanguage,alternate name,isPreferredName,isShortName,isColloquial,isHistoric,from,to
0,2181137,3038886,ca,Pic de Tristaina,1,,,,,
1,13898832,3038899,ca,Tossalet i Vinyals,,,,,,
2,13898833,3038914,ca,Canal de la Tosa,,,,,,
3,13898834,3038995,ca,Bosc del Solobre,,,,,,
4,13898836,3039037,ca,Solà d’Engordany,,,,,,


Bringing shortname and preferred name in the isolanguage column.

In [10]:
# Filter rows where isPreferredName equals 1 (except for all values `en`)
preferred_row = alname_df_all_values_imported[(alname_df_all_values_imported['isPreferredName'] == "1") & (alname_df_all_values_imported['isolanguage'] != "en")].copy()

# Update the isolanguage column to 'preferred'
preferred_row['isolanguage'] = 'preferred'

# Filter rows where isShortName equals 1
shortname_row = alname_df_all_values_imported[alname_df_all_values_imported['isShortName'] == "1"].copy()

# Update the isolanguage column to 'shortname'
shortname_row['isolanguage'] = 'shortname'

# Concatenate the original DataFrame with the preferred row DataFrame
alname_df_all_values_imported_pref_shortname = pd.concat([alname_df_all_values_imported, preferred_row, shortname_row], ignore_index=True)

In [11]:
alname_df_all_values = db.sql("""SELECT geonameid, isolanguage, "alternate name" as alt_name
FROM alname_df_all_values_imported_pref_shortname
""").df()
alname_df_all_values

,geonameid,isolanguage,alt_name
0,3038886,ca,Pic de Tristaina
1,3038899,ca,Tossalet i Vinyals
2,3038914,ca,Canal de la Tosa
3,3038995,ca,Bosc del Solobre
4,3039037,ca,Solà d’Engordany
...,...,...,...
7551309,7729890,shortname,North America
7551310,8354411,shortname,Soviet Union
7551311,8354411,shortname,СССР
7551312,8354411,shortname,USSR


Optional Step: Taking a look at the top languages in the database (Disclaimer in step 3)

In [12]:
db.sql('''
SELECT isolanguage, count(*) lng_count
FROM alname_df_all_values
GROUP BY isolanguage
ORDER BY lng_count DESC
LIMIT 25
''').df()

,isolanguage,lng_count
0,zh,1008392
1,en,728195
2,no,691172
3,es,556830
4,ru,546593
5,fa,517259
6,id,396136
7,ar,304521
8,th,265978
9,ja,196185


Only picking out 10 languages + shortname + preferred

In [13]:
# 10 Languages selected: 'zh', 'en', 'es', 'fr', 'ja', 'pt', 'ru', 'de', 'ar', 'ko'
alname_df_all_values_selected = db.sql('''
SELECT *
FROM alname_df_all_values
WHERE isolanguage in ('zh', 'en', 'es', 'fr', 'ja', 'pt', 'ru', 'de', 'ar', 'ko', 'shortname', 'preferred')
''').df()
alname_df_all_values_selected

,geonameid,isolanguage,alt_name
0,9089920,zh,拿司
1,9089921,zh,素地屋
2,9089923,zh,日阿洞
3,9089925,zh,白扭洽色
4,9089927,zh,强波隔勒
...,...,...,...
4090910,30521,ar,Āl ‘Īsá
4090911,30521,ar,ال عيسى
4090912,30524,ar,الضحاكي
4090913,30527,ar,Ḩijlān


Taking the first alt name by each language
- Geonames have multiple entries for the same language and same location. There could be dozens of alternative names for the same location within the same language.

In [14]:
alname_df_all_values_selected_only_first_loc = db.sql('''WITH RankedRows AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY geonameid, isolanguage) AS row_num
    FROM alname_df_all_values_selected
)
SELECT geonameid, isolanguage, alt_name
FROM RankedRows
WHERE row_num = 1;
''').df()
alname_df_all_values_selected_only_first_loc

,geonameid,isolanguage,alt_name
0,1727,ar,معوبرة
1,1766,ar,خان خيام الوليد
2,1807,ar,خشام
3,1813,ar,عبود الجدعان
4,2028,ar,درة بیداد العلیا
...,...,...,...
3652220,12740426,ru,Bol’shoy Naergen
3652221,12746331,preferred,桶屋町通り
3652222,12746428,zh,鄂城区
3652223,12746467,en,Masjid Mubarak


Choose English as one of the mandatory listed languages.
- If English is not listed for a particular geoname entry, we will not select it.

In [15]:
alname_df_all_values_selected_only_first_loc_en_filtered =  db.sql('''
SELECT t2.*
FROM alname_df_all_values_selected_only_first_loc t2
JOIN (
SELECT geonameid
FROM alname_df_all_values_selected_only_first_loc
WHERE isolanguage = 'en') t1
ON t2.geonameid = t1.geonameid
''').df()
alname_df_all_values_selected_only_first_loc_en_filtered

,geonameid,isolanguage,alt_name
0,12181875,ru,Станция Новгород-Северский
1,12182202,en,Kamkusa Msiska
2,12182211,en,Mwafulirwa Mwambelo
3,12182323,en,Junju
4,12182457,en,Kanyenda Masanja
...,...,...,...
1046753,6578827,zh,第三號隧道
1046754,6586550,zh,Ti-shih-hao Ch’iao
1046755,6589562,zh,省立臺南醫院
1046756,6607371,en,Pushkino


Optional Step: Test how many languages are there for each geoname entry

In [16]:
db.sql('''
SELECT count_lang, count(*) as lang_group_count
FROM (
	SELECT geonameid, count(*) as count_lang
	FROM alname_df_all_values_selected_only_first_loc_en_filtered
	GROUP BY geonameid
	ORDER BY count_lang DESC
)
GROUP BY count_lang
ORDER BY count_lang DESC
''').df()

,count_lang,lang_group_count
0,12,857
1,11,1511
2,10,2407
3,9,4122
4,8,2246
5,7,2372
6,6,2990
7,5,4238
8,4,18787
9,3,55159


Optional Step: Minimum number of required languages
- If you want a tabular output select the max languges in the input language array (which is 10).
- Here as an example we can select geonames with more than 5 alt names

In [17]:
db.sql('''
SELECT t1.*
FROM alname_df_all_values_selected_only_first_loc_en_filtered t1
JOIN (
	SELECT geonameid, count(*) as count_lang
	FROM alname_df_all_values_selected_only_first_loc_en_filtered
	GROUP BY geonameid
) t2
ON t1.geonameid = t2.geonameid
WHERE t2.count_lang > 2
''').df()

,geonameid,isolanguage,alt_name
0,4231976,shortname,Worth
1,4234477,ar,بروكبورت
2,4235954,ar,مقاطعة كلارك
3,4237730,fr,Comté d'Effingham
4,4247802,shortname,Randolph
...,...,...,...
402395,1835329,preferred,대구광역시
402396,1839118,en,Biyangdo Island
402397,1855274,ja,西蒲原郡
402398,1503219,en,Kizhi-Aksy


Optional Step: JSONification of the language and alt_names
- The dictionary represented in a JSON like string

In [18]:
alname_df_all_values_selected_only_first_loc_en_filtered_jsonified  = alname_df_all_values_selected_only_first_loc_en_filtered.groupby('geonameid').apply(lambda x: json.dumps(dict(zip(x['isolanguage'], x['alt_name'])), ensure_ascii=False)).reset_index()
alname_df_all_values_selected_only_first_loc_en_filtered_jsonified.columns = ['geonameid', 'iso_language_alt_names']
alname_df_all_values_selected_only_first_loc_en_filtered_jsonified

,geonameid,iso_language_alt_names
0,14,"{""en"": ""Takht Arreh Do"", ""ar"": ""تخت أرة دو""}"
1,254,"{""en"": ""Pol-e Bala Rud"", ""ar"": ""جسر بالارود""}"
2,301,"{""fr"": ""Shahr-e ziba"", ""en"": ""Shahr-e Ziba""}"
3,470,"{""fr"": ""place Azadi"", ""pt"": ""Praça Azadi"", ""zh..."
4,566,"{""de"": ""Ghar-e-Roodafshan"", ""fr"": ""Ghar-e-Rood..."
...,...,...
683061,12746714,"{""en"": ""Bosiljevo interchange""}"
683062,12746729,"{""en"": ""Ringwood Magistrates Court""}"
683063,12746744,"{""en"": ""Appleton Dock""}"
683064,12746795,"{""ja"": ""葛城地蔵尊"", ""en"": ""Katsuragi Kṣitigarbha"",..."


Converting the dataset in to Python Dictionary for JSON output

In [19]:
alname_df_all_values_selected_only_first_loc_en_filtered_dictionariezed_df  = alname_df_all_values_selected_only_first_loc_en_filtered.groupby('geonameid').apply(lambda x: dict(zip(x['isolanguage'], x['alt_name']))).reset_index()
alname_df_all_values_selected_only_first_loc_en_filtered_dictionariezed_df.columns = ['geonameid', 'iso_language_alt_names']

alname_df_all_values_selected_only_first_loc_en_filtered_dict = alname_df_all_values_selected_only_first_loc_en_filtered_dictionariezed_df.set_index('geonameid')['iso_language_alt_names'].to_dict()

### Testing out the different process dataset format

Test 1: Dataset output

In [20]:
print(alname_df_all_values_selected_only_first_loc_en_filtered[alname_df_all_values_selected_only_first_loc_en_filtered["geonameid"]==1269750].to_markdown())

|        |   geonameid | isolanguage   | alt_name    |
|-------:|------------:|:--------------|:------------|
|  62447 |     1269750 | fr            | Inde        |
| 111100 |     1269750 | ar            | الهند       |
| 245786 |     1269750 | ru            | Индия       |
| 278989 |     1269750 | es            | Unión India |
| 314823 |     1269750 | zh            | 印度        |
| 444853 |     1269750 | preferred     | Ινδία       |
| 581391 |     1269750 | ja            | インド      |
| 614529 |     1269750 | en            | India       |
| 659641 |     1269750 | ko            | 인도        |
| 817618 |     1269750 | de            | Indien      |
| 993566 |     1269750 | pt            | Índia       |


Test 2: JSONified table output

In [21]:
db.sql('''
SELECT *
FROM alname_df_all_values_selected_only_first_loc_en_filtered_jsonified
WHERE geonameid==1269750
''')

┌───────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ geonameid │                                          iso_language_alt_names                                          │
│   int32   │                                                 varchar                                                  │
├───────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│   1269750 │ {"fr": "Inde", "ar": "الهند", "ru": "Индия", "es": "Unión India", "zh": "印度", "preferred": "Ινδία", …  │
└───────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────────┘

Test 3: Dictionary (/JSON) output

In [22]:
alname_df_all_values_selected_only_first_loc_en_filtered_dict[1269750]

{'fr': 'Inde',
 'ar': 'الهند',
 'ru': 'Индия',
 'es': 'Unión India',
 'zh': '印度',
 'preferred': 'Ινδία',
 'ja': 'インド',
 'en': 'India',
 'ko': '인도',
 'de': 'Indien',
 'pt': 'Índia'}

### Saving the files

The dataset in CSV and the JSON in JSON is available in the zip file `geoname_alt_names.zip`

In [23]:
alname_df_all_values_selected_only_first_loc_en_filtered.to_csv("geoname_alt_names.csv", index=False, encoding='utf-8')

In [24]:
with open("geoname_alt_names.json", "w", encoding='utf-8') as file:
    json.dump(alname_df_all_values_selected_only_first_loc_en_filtered_dict, file, ensure_ascii=False)